In [46]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np

url_headers = {
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Encoding':'gzip, deflate, br',
    'Accept-Language':'es-ES,es;q=0.9',
    'Cache-Control':'max-age=0',
    'Sec-Ch-Ua':'"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    'Sec-Ch-Ua-Mobile':'?0',
    'Sec-Ch-Ua-Platform':'"Windows"',
    'Sec-Fetch-Dest':'document',
    'Sec-Fetch-Mode':'navigate',
    'Sec-Fetch-Site':'same-origin',
    'Sec-Fetch-User':'?1',
    'Upgrade-Insecure-Requests':'1',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

lista_distritos = ['bailen-miraflores', 'campanillas/campanillas', 'carretera-de-cadiz',
                  'centro', 'churriana', 'ciudad-jardin', 'cruz-de-humilladero', 'este', 'martiricos-la-roca', 
                  'puerto-de-la-torre', 'teatinos'
                  ]




### SACAMOS LOS PISOS DEL DISTRITO SELECCIONADO, PAGINA POR PAGINA, Y GUARDAMOS EL RESULTADO EN UN .CSV

for distrito in lista_distritos:

    list_df_distrito = []
    cols = ['Título', 'Precio', 'Garaje', 'Descripción', 'Detalles']
    keep_turning_page = True
    url = "https://www.idealista.com/venta-viviendas/malaga/" + distrito + '/'

    while keep_turning_page:

        response = requests.get(url, headers = url_headers)
        print(response)

        html_cont = response.content
        soup = bs(html_cont, 'lxml')        

        all_30 = soup.find_all('div', attrs={'class': "item-info-container"})

        for ad in all_30:
            list_row = []
            list_row.append(ad.find('a', attrs={'role': "heading"}).text)                       # 1 - titulo anuncio
            list_row.append(ad.find('span', attrs={'class': "item-price h2-simulated"}).text)   # 2 - precio
            if ad.find('span', attrs={'class': "item-parking"}) != None:
                list_row.append(ad.find('span', attrs={'class': "item-parking"}).text)          # 3 - garaje/no garaje
            else:
                list_row.append(None)
            if ad.find('p', attrs={'class': "ellipsis"}) != None:                               # 4 - descripción
                list_row.append(ad.find('p', attrs={'class': "ellipsis"}).text)
            else:
                list_row.append(None)                     
            details = ad.find_all('span', attrs={'class': "item-detail"})    
            details_list = []                    
            for detail in details:                                                              # 5 - nº habs, m2, planta y ascensor/no ascensor
                details_list.append(detail.text)
            list_row.append(details_list)

            new_row = pd.DataFrame([list_row], columns = cols)
            list_df_distrito.append(new_row)

        # Navegamos a la siguiente página
            
        next_page_endpoint = soup.find('a', attrs={'class': "icon-arrow-right-after"})
        
        if next_page_endpoint != None:
            url = 'https://www.idealista.com' + next_page_endpoint['href']
        else:
            keep_turning_page = False

    df = pd.concat(list_df_distrito, ignore_index=True) 
    distrito_filename = distrito.replace('/','-')
    df.to_csv('./idealista_Malaga_' + distrito_filename + '.csv')    

     

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [10]:
### JUNTAMOS TODOS LOS CSVs DE DISTRITOS EN UN SOLO DATAFRAME Y LO GUARDAMOS

import os
import pandas as pd

lista_csvs = []

folder_path = './DB idealista VENTA'
file_list = os.listdir(folder_path)
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    data = pd.read_csv(file_path, index_col=0)
    distrito = file_name.replace('.csv','').split(sep='_')[-1]
    if distrito == 'campanillas-campanillas':
        distrito = 'campanillas'
    data['Distrito'] = distrito.capitalize()
    lista_csvs.append(data)
        
df = pd.concat(lista_csvs, ignore_index=True)
df.to_csv('./DB_idealista_VENTA_completo.csv')

df



,Título,Precio,Garaje,Descripción,Detalles,Distrito
0,"\nPiso en calle Nicolás Salmerón, Victoria Eug...",135.000€,NaN,\n¡Haz realidad tu propósito de año nuevo! Est...,"['2 hab.', '59 m²', 'Planta 1ª exterior sin as...",Bailen-miraflores
1,"\nPiso en Victoria Eugenia, Málaga\n",199.000€,NaN,\nTe presentamos este piso que SE ENTREGARIA T...,"['3 hab.', '81 m²', 'Planta 5ª exterior con as...",Bailen-miraflores
2,"\nPiso en calle Antonio Jiménez Ruiz, Los Cast...",175.000€,NaN,\nExclusivo piso ubicado en el barrio de La Tr...,"['2 hab.', '62 m²', 'Planta 2ª interior con as...",Bailen-miraflores
3,"\nPiso en C. Eugenio Gross, 11, Los Castillejo...",342.200€,Garaje incluido,"\nBerkshire Hathaway HomeServices, líder recon...","['2 hab.', '104 m²', 'Planta 4ª exterior con a...",Bailen-miraflores
4,"\nPiso en calle Ecuador, s/n, Los Castillejos ...",175.000€,NaN,\nOPORTUNIDAD.\nTe presentamos este encantador...,"['3 hab.', '83 m²', 'Planta 2ª exterior con as...",Bailen-miraflores
...,...,...,...,...,...,...
4680,\nChalet adosado en El Tejar - Hacienda Bizcoc...,479.000€,NaN,\nAYRE ESTATES LE OFRECE EN EXCLUSIVA LA VENTA...,"['4 hab.', '150 m²']",Teatinos
4681,\nCasa o chalet independiente en calle Gaspar ...,175.000€,NaN,"\nSe vende loft moderno, increíble ubicación, ...","['2 hab.', '80 m²']",Teatinos
4682,"\nPiso en El Consul - El Romeral, Málaga\n",395.000€,Garaje incluido,\nInternational Inmobiliaria te presenta esta ...,"['4 hab.', '125 m²', 'Planta 2ª exterior con a...",Teatinos
4683,"\nDúplex en calle Doctor Norman Bethune, 5, El...",189.500€,Garaje incluido,"\nVenta dúplex, semi nuevo en perfecto estado ...","['4 hab.', '100 m²', 'Planta 2ª exterior con a...",Teatinos
